## Inventary


#### admin acount required

In [ ]:
from arcgis.gis import GIS
gis = GIS("http://www.arcgis.com/","rmartin_esri_colombia","***")
import pandas as pd
from datetime import datetime
from time import sleep

In [ ]:
gis.admin.credits.credits

#### Inventary

In [ ]:
columns = ["Folder","Owner", "Title", "Name", "Type", "Views", 
           "Size (mb)", "Creation date", "Edit date", "Item ID","Acces","Url"]
def getInventaryItems(user):    
    datos = []
    foldersName = [None]+[folder["title"] for folder in user.folders]
    for folder in foldersName:
        items = user.items(folder)
        if folder is None:
            folder = "Root"
        for item in items:
            datos.append([folder, 
                          item.owner, 
                          item.title, 
                          item.name, 
                          item.type, 
                          item.numViews, 
                          item.size/1048576 , 
                          datetime.utcfromtimestamp(item.created/1000).strftime('%Y-%m-%d %H:%M:%S'),
                          datetime.utcfromtimestamp(item.modified/1000).strftime('%Y-%m-%d %H:%M:%S'),
                          item.id,
                          item.access,
                          item.url])
    return datos   

**Note: Executing the following cell may take a few minutes.**

In [ ]:
df_i = pd.DataFrame()

##limit the users
for user in gis.users.search():
    datos = getInventaryItems(user)
    df = pd.DataFrame(datos, columns=columns)
    df_i=pd.concat([df_i,df])


In [ ]:
df.dtypes

In [ ]:
list(df.select_dtypes(include=['float64','int64']))

In [ ]:
df_i.to_csv('inventary.csv',encoding='utf8',sep=';',index=False)

In [ ]:
print("Total almacenamiento de {} Mb == {:.2f} Gb".format(df['Size (mb)'].sum(),df['Size (mb)'].sum()))

In [ ]:
# para mostrar muchos registros
pd.set_option('display.max_rows', 500)

In [ ]:
len(df_i['Propietario'].unique())

In [ ]:
df_1 = pd.read_csv('temp.csv',encoding='utf8',sep=';')

In [ ]:
df_1

### Calculate credits : https://doc.arcgis.com/en/arcgis-online/administer/credits.htm NOT already

In [ ]:
df_i.groupby(list(df_i)).sum().sort_values(by='Size (mb)',ascending=False).head(10)

# Por tipo

In [ ]:
# weight by type
types=df_i.groupby(['Tipo']).sum().reset_index()
w = {t:float(types.loc[(types['Tipo'] == t), 'Tamaño (mb)']) for (t) in list(types.Tipo.unique())}
types

### Per type

In [ ]:
#by owner
owner=df_i.groupby(['Propietario']).sum().sort_values('Tamaño (mb)',ascending=False)
owner

In [ ]:
#by owner and type
owner_type=df_i.groupby(['Propietario','Tipo']).sum().sort_values('Tamaño (mb)',ascending=False)
owner_type

In [ ]:
#by owner and folder
owner_folder=df_i.groupby(['Propietario','Folder']).sum().sort_values('Tamaño (mb)',ascending=False)
owner_folder.reset_index()

In [ ]:
owner_item=df_i.groupby(['Propietario','Título','Tipo','Item ID','Fecha creación','Fecha modificación']).sum().sort_values('Número de vistas',ascending=False).reset_index()
owner_item

In [ ]:
#by owner
df_i.query("Propietario == 'mtoro_geek'").sort_values('Tamaño (mb)',ascending=False)

In [ ]:
#  by type
df_i[df_i['Tipo']=='Service Definition'].sort_values('Tamaño (mb)',ascending=False)

# vistas

In [ ]:
df_i.groupby(['Título']).sum().reset_index()

### save

In [ ]:
# df_i.to_csv("inventary.csv",encoding='utf-8-sig')
df_i.groupby(['Tipo']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/type.csv",encoding='utf-8-sig',sep=';',decimal=',')
df_i.groupby(['Propietario']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/owner.csv",encoding='utf-8-sig',sep=';',decimal=',')
df_i.groupby(['Propietario','Tipo']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/owner_type.csv",encoding='utf-8-sig',sep=';',decimal=',')
df_i.groupby(['Propietario','Folder']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/owner_folder.csv",encoding='utf-8-sig',sep=';',decimal=',')
df_i.groupby(['Propietario','Título','Item ID','Tipo','Fecha creación','Fecha modificación']).sum().sort_values('Tamaño (mb)',ascending=False).reset_index().to_csv("inventario_semillero/owner_item.csv",encoding='utf-8-sig',sep=';',decimal=',')
#download in Files tab; and click on file

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('inventary.csv',sep=';')
df.dtypes

In [ ]:
df.to_parquet('inventary.parquet.gzip',
              compression='gzip')  

In [ ]:
!pip install pyarrow